# Семинар по деревьям

Берем данные boston

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
import re
from sklearn.cross_validation import train_test_split

In [ ]:
boston = load_boston()

In [ ]:
boston.keys()

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(boston["data"], boston["target"])

In [ ]:
data_train = pd.DataFrame(X_tr, columns=boston["feature_names"])
data_test = pd.DataFrame(X_te, columns=boston["feature_names"])

In [ ]:
data_train.head()

In [ ]:
data_train.shape

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

Далее мы будем реализовывать первый шаг в построении решающего дерева - выбор признака и порога для разделения в корне дерева. Мы будем писать максимально понятный, но не оптимальный, код, т. е. в sklearn все реализовано более эффективно.

In [ ]:
# чтобы было удобно сортировать объекты вместе с целевым вектором, допишем его в датафрейм


In [ ]:
# чтобы было удобно перебирать порог на первый признак, сортируем датафрейм по нему


In [ ]:
data_train.head()

In [ ]:
# считаем качество - дисперсию ответов в левом и правом поддереве с весами, как в лекции


In [ ]:
# рисуем график порог - качество разделения


Чем меньше, тем лучше, поэтому оптимум остигается где-то в CRIM = 7.

Теперь сделаем это для каждого признака (copypaste - это очень плохо, но для демонстрации хорошо).

Обратите внимание: чтобы было удобно сравнивать значение критерия для разных признаков, мы все рисуем на одном графике. Но шкала (множество значений) у каждого признака своя. Так что мы будем откладывать по оси x просто числа от 0 до длины выборки, и величину оптимального порога по графику будет определить нельзя. По графику мы сможем определить только оптимальный признак для разделения.

In [ ]:
for feat in data_train.columns[:-1]:
    # code for one feature here
plt.legend(loc=(1, 0))

Кажется, выиграл RM. Нарисуем для него график отдельно (уже с его осью): 

Величина порога:

In [ ]:
thresh = data_train[feat][np.argmin(quals)]
print(thresh)

Нарисуем выборку в осях RM - target и изобразим порог, по которому мы разделили

In [ ]:
plt.scatter(data_train["RM"], data_train["target"])
plt.plot([thresh, thresh], [0, 60], color="red")

Видно, что точки справа от красной линии лежат почти все выше 30, а слева - ниже, т. е. этот признак действительно очень хорошо разделяет выборку.

## Критерий останова построения дерева

Очевидно, для любой обучающей выборки можно построить решающее дерево, которое имеет нулевую ошибку на данной выборке. Однако в этом случае имеет место **переобучение**.

В связи с этим встаёт вопрос: в каком случае вершину следует объявить листовой?

Рассмотрим модельную задачу регрессии. Объектами будут являться точки на плоскости (т.е. каждый объект описывается 2 признаками), целевой переменной — расстояние от объекта до точки (0, 0).

In [ ]:
import numpy as np
import pandas as pd
import pylab as plt
%matplotlib inline

In [ ]:
def get_grid(data):
    x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
    y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
    return np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))

In [ ]:
data_x = np.random.normal(size=(100, 2))
data_y = (data_x[:, 0] ** 2 + data_x[:, 1] ** 2) ** 0.5
plt.figure(figsize=(8, 8))
plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=100, cmap='spring')

#### 1. Все объекты в вершине относятся к одному к классу.

    Простое условие, но приводит к переобучению.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
clf = DecisionTreeRegressor()
clf.fit(data_x, data_y)

xx, yy = get_grid(data_x)

predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

plt.figure(figsize=(8, 8))
plt.pcolormesh(xx, yy, predicted, cmap='spring')
plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=100, cmap='spring')

#### 2. При дальнейшем разбиении выборки глубина дерева превысит допустимую максимальную глубину.
#### 3. При дальнейшем разбиении количество объектов в листьях станет меньше допустимого порога.

In [ ]:
plt.figure(figsize=(14, 14))
for i, max_depth in enumerate([2, 4, None]):
    for j, min_samples_leaf in enumerate([1, 5, 15]):
        clf = DecisionTreeRegressor(max_depth=max_depth, min_samples_leaf=min_samples_leaf)
        clf.fit(data_x, data_y)
        xx, yy = get_grid(data_x)
        predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
        
        plt.subplot2grid((3, 3), (i, j))
        plt.pcolormesh(xx, yy, predicted, cmap='spring')
        plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=30, cmap='spring')
        plt.title('max_depth=' + str(max_depth) + ', min_samples_leaf: ' + str(min_samples_leaf))

Можно увидеть, что увеличение глубины дерева и уменьшение количества объектов в листьях способствует гибкости модели и, как следствие, переобучению. 

#### 4. Стрижка деревьев (pruning)
Все предыдущие критерии останова были направлены против построения переобученного дерева. При использовании прунинга сначала строится максимально переобученное дерево, после чего оно усовершенствуется путем удаления некоторых листовых вершин. Данный способ работает лучше, чем рассмотренные ранее, однако является более трудоёмким.

# Неустойчивость решающих деревьев

Решающие деревья — это алгоритмы, неустойчивые к изменениям обучающей выборки, т.е. при малейших её изменениях итоговый классификатор может радикально измениться.


In [ ]:
plt.figure(figsize=(20, 6))
for i in range(3):
    clf = DecisionTreeRegressor(random_state=42)

    indecies = np.random.randint(data_x.shape[0], size=(data_x.shape[0] * 0.9))
    clf.fit(data_x[indecies], data_y[indecies])
    xx, yy = get_grid(data_x)
    predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    plt.subplot2grid((1, 3), (0, i))
    plt.pcolormesh(xx, yy, predicted, cmap='winter')
    plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=30, cmap='winter')

# Решающие леса

Чтобы уменьшить влияние рассмотренных недостатков решающих деревьев, используют **случайные леса (random forest)**. Одно дерево может ошибаться, поэтому давайте построим много деревьев и "усредним" их ответы.

Если строить все эти леса по одной и той же обучающей выборке, они будут одинаковые, поэтому для каждого решающего дерева, которое мы строим:
* вместо исходной обучающей выборки $X^l$ используется бутстрап (т.е. выборка $l$ объектов **с возвращениями**);
* в каждой внутренней вершине выбор условия осуществляется не среди всех признаков, а среди $q$ случайно выбранных.

Такая рандомизация обеспечит нам различность деревьев. После того, как таким образом были получены алгоритмы $b_1(x), ... b_N(x)$ можно построить итоговый алгоритм как:
* **выбор большинства** в случае классификации: $a(x) = \arg \max_{y in \mathbb{Y}} \sum_{n=1}^N [b_n(x) = y]$
* **среднее** в случае регрессии: $a(x) = \frac{1}{N} \sum_{n = 1}^N b_n(x)$

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
plt.figure(figsize=(20, 6))
for i, n in enumerate([1, 2, 10]):
    clf = RandomForestRegressor(n_estimators=n, random_state=42)
    clf.fit(data_x, data_y)
    xx, yy = get_grid(data_x)
    
    predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    plt.subplot2grid((1, 3), (0, i))
    
    plt.pcolormesh(xx, yy, predicted, cmap='autumn')
    
    plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=30, cmap='autumn')
    plt.title('estimators: %i' % n)

plt.show()